In [1]:
import pandas as pd
import numpy as np
import os
from filtering_utils import DS_PATH
%matplotlib inline

pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

In [2]:
TRACKS_PATH = os.path.join(DS_PATH, 'tracks_filtered_dummies.csv')
tracks = pd.read_csv(TRACKS_PATH, index_col=0)
tracks.head(10)

,split,subset,Ambient,Ambient Electronic,Avant-Garde,Electroacoustic,Electronic,Experimental,Experimental Pop,Folk,Hip-Hop,Instrumental,Lo-Fi,Noise,Pop,Punk,Rock,Soundtrack
track_id,,,,,,,,,,,,,,,,,,
2,training,small,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,training,medium,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,training,small,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
10,training,small,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
20,training,large,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
26,training,large,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
30,training,large,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
46,training,large,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
48,training,large,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [3]:
AUDIO_PATH = os.path.join(os.getcwd(), 'fma_medium')

viable_tracks_ids = []
for dir_el in os.listdir(AUDIO_PATH):
    if dir_el in ('checksums', 'README.txt'): continue
    for track_filename in os.listdir(os.path.join(AUDIO_PATH, dir_el)):
        while track_filename.startswith('0'):
            track_filename = track_filename[1:]
        viable_tracks_ids.append(int(track_filename.removesuffix('.mp3')))
viable_tracks_ids

[2,
 3,
 5,
 10,
 134,
 136,
 139,
 140,
 141,
 148,
 181,
 182,
 190,
 193,
 194,
 197,
 198,
 200,
 203,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 237,
 247,
 249,
 255,
 256,
 257,
 258,
 341,
 343,
 352,
 368,
 369,
 398,
 399,
 400,
 405,
 414,
 424,
 425,
 458,
 459,
 512,
 534,
 535,
 536,
 537,
 540,
 546,
 550,
 564,
 573,
 583,
 584,
 585,
 590,
 591,
 592,
 593,
 602,
 603,
 605,
 608,
 615,
 619,
 620,
 621,
 625,
 629,
 631,
 633,
 664,
 665,
 666,
 667,
 668,
 669,
 672,
 673,
 674,
 676,
 677,
 679,
 680,
 681,
 684,
 686,
 690,
 694,
 695,
 204,
 397,
 574,
 635,
 696,
 737,
 758,
 814,
 861,
 698,
 701,
 702,
 703,
 704,
 705,
 706,
 707,
 708,
 709,
 714,
 715,
 716,
 717,
 718,
 720,
 734,
 735,
 736,
 738,
 739,
 740,
 741,
 742,
 743,
 744,
 745,
 747,
 748,
 749,
 750,
 751,
 752,
 753,
 754,
 755,
 756,
 757,
 759,
 760,
 761,
 762,
 763,
 764,
 765,
 766,
 767,
 768,
 769,
 770,
 771,
 772,
 773,
 774,
 775,
 776,
 777,
 819,
 820,
 821,
 822,
 823,
 824,
 825,

In [4]:
assert set(viable_tracks_ids).intersection(tracks.index) == set(tracks.index).intersection(viable_tracks_ids)
viable_tracks_ids = list(set(viable_tracks_ids).intersection(tracks.index))
print(tracks.shape)
tracks = tracks.loc[viable_tracks_ids]
tracks.shape

(98988, 18)


(21924, 18)

In [5]:
assert all(idx in viable_tracks_ids for idx in tracks.index)

In [6]:
tracks.reset_index(inplace=True)
tracks.head()

,track_id,split,subset,Ambient,Ambient Electronic,Avant-Garde,Electroacoustic,Electronic,Experimental,Experimental Pop,Folk,Hip-Hop,Instrumental,Lo-Fi,Noise,Pop,Punk,Rock,Soundtrack
0,2,training,small,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,3,training,medium,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,5,training,small,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,10,training,small,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,131119,training,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [7]:
train_ids = tracks[tracks['split'] == 'training']['track_id']
val_ids = tracks[tracks['split'] == 'validation']['track_id']
test_ids = tracks[tracks['split'] == 'test']['track_id']
train_ids.shape, val_ids.shape, test_ids.shape

((17498,), (2193,), (2233,))

In [8]:
train_ids

0             2
1             3
2             5
3            10
4        131119
5        131122
6        131126
7        131127
8        131128
9        131146
10       131153
11       131161
12       131164
13       131166
14       131171
15       131172
16       131177
17       131178
18       131179
19       131180
20       131189
21       131190
22       131191
23       131192
24       131193
25       131194
26       131195
27       131196
28       131197
29          134
30          136
31          139
32          140
33          141
37          190
38          193
39          194
40          197
41          198
42          200
43          203
44          204
45          207
46          208
47          209
48          210
49          211
50          212
51          213
52       131292
53       131303
54          247
55          249
56       131323
57       131324
58       131325
59       131326
60          255
61          256
62          257
63          258
64       131327
65      

In [9]:
N_FFT = 2048
HOP_LENGTH = 512
SAMPLING_RATE = 22050
SPLIT_DURATION = 3.0
REPOPULATE = True

In [11]:
label_columns = tracks.drop(['split', 'subset', 'track_id'], axis=1).columns
label_columns

Index(['Ambient', 'Ambient Electronic', 'Avant-Garde', 'Electroacoustic',
       'Electronic', 'Experimental', 'Experimental Pop', 'Folk', 'Hip-Hop',
       'Instrumental', 'Lo-Fi', 'Noise', 'Pop', 'Punk', 'Rock', 'Soundtrack'],
      dtype='object')

In [12]:
import multiprocessing as mp
from tqdm.notebook import tqdm
from spectrograms import generate_track_melspectrograms, pickle_spectrograms_set
tqdm.pandas()


def create_spectrograms_for_set(set_tracks_ids: pd.Index, split_duration: float = SPLIT_DURATION, max_shape: tuple[int, int] = (0, 0)) -> tuple[np.ndarray, np.ndarray]:
    X = []
    y = []
    set_tracks = tracks.loc[set_tracks_ids, 'track_id']
    
    with mp.Pool(os.cpu_count() - 1) as pool:
        for idx, track_id in tqdm(set_tracks.items(), total=len(set_tracks)):
            track_spectrograms = pool.apply(generate_track_melspectrograms, args=(track_id, split_duration,))

            if not isinstance(track_spectrograms, np.ndarray):
                continue
            
            for track_spectrogram in track_spectrograms:
                if track_spectrogram.shape > max_shape:
                    max_shape = track_spectrogram.shape
                X.append(track_spectrogram)
            
            track_genre = tracks.loc[idx, label_columns]
            for _ in range(len(track_spectrograms)):
                y.append(track_genre)
        
        for idx, track_spectrogram in enumerate(X):
            if track_spectrogram.shape == max_shape: continue
            X[idx] = np.resize(track_spectrogram, max_shape)
            
    return np.array(X, dtype=np.float16), np.array(y, dtype=np.float16), max_shape

In [23]:
x_train, y_train, max_shape = create_spectrograms_for_set(train_ids.index)
x_train.shape, y_train.shape

  0%|          | 0/17498 [00:00<?, ?it/s]

[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/layer3.c:INT123_do_layer3():1841] error: dequantization failed!
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!
[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!
[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!
[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!
[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!
[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!
[src/libmpg123/layer3.c:INT123_do_layer3():1841] error: dequantization failed!
[src/libmpg123/parse.c:do_readahead():1099] warning: 

((181982, 128, 130), (181982, 16))

In [24]:
pickle_spectrograms_set(x_train, y_train, 'train')

In [25]:
del x_train, y_train

In [15]:
x_val, y_val, _ = create_spectrograms_for_set(val_ids.index, max_shape=max_shape)
x_val.shape, y_val.shape

  0%|          | 0/2193 [00:00<?, ?it/s]

[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...


((22848, 128, 130), (22848, 16))

In [16]:
pickle_spectrograms_set(x_val, y_val, 'validation')

In [17]:
del x_val, y_val

In [20]:
x_test, y_test, max_shape = create_spectrograms_for_set(test_ids.index, max_shape=max_shape)
x_test.shape, y_test.shape

  0%|          | 0/2233 [00:00<?, ?it/s]

[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!


((23299, 128, 130), (23299, 16))

In [21]:
pickle_spectrograms_set(x_test, y_test, 'test')

In [22]:
del x_test, y_test